# Bonus Time

We'll start with the agent we built in ex 1, but add a UiPath-hosted MCP server, so the agent can invoke a MCP tool.

There is an MCP Server created in AMER Presales folder in staging.uipath.com/uipathlabs - the URL for this srever is: https://staging.uipath.com/uipathlabs/Playground/agenthub_/mcp/4332993a-91e0-4b3c-84ca-c4b47bf6e386/uipath-genai-websummary

The MCP server contains one tool - the GenAI Activities Web Summary activity.

## First things first

Let's run some imports for libraries and set up our LLM to go through the UiPath trust layer again.

In [1]:
import os
from typing import Any
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain.agents import create_agent
from uipath_langchain.chat.models import UiPathChat
from langchain_mcp_adapters.tools import load_mcp_tools
from mcp import ClientSession
from langchain_mcp_adapters.client import MultiServerMCPClient

llm = UiPathChat(
    model="gpt-4o-2024-11-20",
    temperature=0,
    max_tokens=4000,
    timeout=30,
    max_retries=2,
    # other params...
)

llm.invoke("What is the capital of the moon?")

Response content: {"id":"chatcmpl-Cz83MjkpfOfTr8AbJYdv1Wz5OGYqu","model":"gpt-4o-2024-11-20","object":"chat.completion","choices":[{"finish_reason":"stop","index":0,"message":{"content":"The Moon does not have a capital, as it is not a country or governed by any political entity. It is a natural satellite of Earth. However, humans have explored the Moon, and notable locations include the Apollo landing sites, such as Tranquility Base, where Apollo 11 landed in 1969.","role":"assistant"}}],"created":1768685000,"usage":{"completion_tokens":64,"prompt_tokens":15,"total_tokens":79,"cache_read_input_tokens":0}}



AIMessage(content='The Moon does not have a capital, as it is not a country or governed by any political entity. It is a natural satellite of Earth. However, humans have explored the Moon, and notable locations include the Apollo landing sites, such as Tranquility Base, where Apollo 11 landed in 1969.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 15, 'total_tokens': 79, 'cache_read_input_tokens': 0}, 'model_name': 'gpt-4o-2024-11-20', 'finish_reason': 'stop', 'system_fingerprint': 'chatcmpl-Cz83MjkpfOfTr8AbJYdv1Wz5OGYqu', 'created': 1768685000}, id='lc_run--019bcdd7-561c-7281-bbce-b80dfd0337a4-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 15, 'output_tokens': 64, 'total_tokens': 79})

# Now we'll get right to business

We'll define a simple agent like before, but this time wrap the agent with an MCP client. 

This follows a simple sequence:

* Connect to MCP server
* Read the available tools
* Add those tools to the agent
* Invoke the agent

MCP allows us to decouple the tools definition from the agent or worflow code that runs the agent - this means that I can change the tools on the server without changing the code we're writing here. This separation speeds up development, since teams can independently build tool catalogs, while agent engineers can simply discover and add them to agents.

### Connect to the MCP Server with a client

The below code creates the connection so the client can discover which tools are available.

In [3]:
client = MultiServerMCPClient(
    {
        "weather": {
            "transport": "http",
            "url": "https://staging.uipath.com/uipathlabs/Playground/agenthub_/mcp/4332993a-91e0-4b3c-84ca-c4b47bf6e386/uipath-genai-websummary",
            "headers": {  
                "Authorization": "Bearer " + os.getenv("UIPATH_ACCESS_TOKEN"),  
            },  
        }
    }
)

tools = await client.get_tools()
print(tools)

[StructuredTool(name='webSummary', description='Generate AI-based summaries of web search results, synthesizing insights from multiple sources with citations.', args_schema={'type': 'object', 'properties': {'query': {'type': 'string', 'title': 'Search', 'description': 'The natural language query to search the web for'}}, 'additionalProperties': False, 'required': ['query']}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x7d2be5074ae0>)]


### Create the agent with the tools

In [9]:
from langchain.messages import HumanMessage

agent = create_agent(llm, tools=tools)

### Invoke the agent

result = await agent.ainvoke({"messages": [HumanMessage(content="Tell me the latest in AI Agent news")]})
print(result["messages"][-1].content)


Here are some recent developments in AI agents:

1. **LiveRamp's Agentic Orchestration Launch** (October 2025): LiveRamp introduced a platform enabling autonomous AI agents to access identity resolution and audience activation tools, supporting marketing operations. This comes amid a significant rise in agentic AI job postings.

2. **G2's AI Roundup**: Highlights include OpenAI's partnership with Bain & Company for B2B applications, ChatGPT's Search rollout for real-time information retrieval, and advancements in MarTech like Zeta Global's acquisition of LiveIntent for identity resolution.

3. **Microsoft's Copilot Studio**: Microsoft announced the public preview of Copilot Studio, which allows users to create autonomous agents. This initiative is scaling into 2025.

4. **C.H. Robinson's AI Agent Fleet Expansion**: The company expanded its fleet of AI agents, automating over 3 million tasks such as quoting and order processing.

5. **Enterprise Adoption Trends**: By 2025, 45% of enterp

# Your challenge

Take this agent and add structured input/output with the graph approach that we used in the main exercise.

# Extra Bonus

Coding agents are continuously improving and have more recently proven to be able to write high quality code across large codebases. 

When using coding agents inside Cursor, Antigravity, Claude Code etc. - the most important rule to remember is that it operates like any other agent: it has an LLM, and it uses tools to gather context before taking actions. The critical piece is **context**.

LLMs are great at writing code that they have seen before (i.e. code samples that were included in their pre-training, or as part of their fine-tuning / RL). This means that newer code, such as new SDKs that UiPath is releasing is not inherently known to the LLMs that power coding agents. 

# Builing UiPath Coded Agents with Coding Agents

As the developer, you'll likely need to take extra steps to provide the right context to coding agents in order for them to accurately and precisely code the UiPath coded agent.

Fortunately, our P&E team has built some convenient context tools in the Python SDK. 

## Start a new project

1. Open a new terminal (outside of Cursor or your IDE)
2. Make a new directory `mkdir vibe-coding-agents` and navigate into it `cd vibe-coding-agents`
3. Initialize a project with uv `uv init . --python 3.12`
4. Create virtual environment `uv venv && source .venv/bin/activate` (Mac/Linux) or `uv venv && .venv/Scripts/activate` (Windows)
5. Install uipath-langchain `uv add uipath-langchain`
6. Create a new uipath agent `uipath new my-vibe-coded-agent`
7. Authenticate `uipath auth` with --staging if you are authenticating to staging
8. Initialize `uipath init`

## Open your IDE (Cursor, Antigravity etc.)

Explore the files in the project. Take special note of Agents.md and Claude.md.

Claude.md is just a pointer to Agents.md. Claude.md is a markdown file that is by default loaded as context to Claude Code - so if you use Claude code, it will jump to reference Agents.md. Similarly, Agents.md is a markdown file that OpenAI Codex uses by default. In Cursor, you can tell the agent to look at these files in the settings.

The real magic is when we look at what Agents.md contains and how it references 3 more markdown files located in the .agents folder.

CLI_REFERENCE.md
REQUIRED_STRUCTURE.md
SDK_REFERENCE.md

Explore the contents of these files.

Note how they provide very helpful reference info and sample code - this is amazing context to provide the coding agent when prompting. 

In Cursor, you can tell the agent to focus on these instructions explicitly using `@` and then selecting the files you want to include in the prompt. 

## Try it out

In your new project, try a prompt like the below (just illustrative to show how we explicitly reference the markdown files - feel free to change the prompt or improve)

'''
refer to @.agent/REQUIRED_STRUCTURE.md , let's update the agent in @main.py to be a UiPath coded agent that uses a math tool. Create a math tool to perform typical arithmetic operations.
Update the GraphState structure to represent an input math problem and the GraphOutput to output the answer and an explanation as two separate keys. 
The agent should solve the problem with the tool and provide a detailed written explanation.
Update the StateGraph accordingly.
'''

Next ask the coding agent to write an input.json file

'''create a sample input JSON in a new file called input.json. 
SHould be in the structure according to the Input you created.'''

## Run the agent

Open a terminal in your IDE.

`uipath run agent --file input.json`

## Don't expect it to work!

Expect issues - if you run into errors, ask the coding agent to help you solve them.

## Example output

Your agent may produce something different. Example output below

<div style="text-align:center">
<img src="assets/vibes.png" width="700">
</div>